In [22]:
import pandas as pd

import numpy as np

from sklearn.decomposition import PCA

import plotly.graph_objects as go
import plotly.express as px


In [7]:
filenames = ['data/scaled_dataset.csv', 'data/pca_without_leak_dataset.csv', 'data/merged_dataset.csv']

In [70]:
datasets = []

for filename in filenames:
    datasets.append(pd.read_csv(filename))


c:\Users\niels\anaconda3\envs\data2\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning:

Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.



In [9]:
datasets[0].shape, datasets[1].shape, datasets[2].shape

((1079565, 20), (1079565, 15), (1079565, 26))

In [18]:
for col in datasets[2].columns:
    if col not in datasets[0].columns:
        print(col)

UTC_TIME
MSN
Flight
DATE
TIME
FLIGHT


In [71]:
import pandas_profiling

profilereport = pandas_profiling.ProfileReport(scaled_data)
profilereport.to_file("Scaled_data_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [72]:
scaled_data = datasets[0]

In [74]:
list_og_columns = scaled_data.columns

In [75]:
for column in scaled_data.columns:
    scaled_data[f'{column}_diff'] = scaled_data[column].diff()
scaled_data.dropna(inplace=True)
scaled_data.drop(columns=list_og_columns, inplace=True)

In [77]:
scaled_data.columns

Index(['FUEL_USED_2_diff', 'FUEL_USED_3_diff', 'FUEL_USED_4_diff',
       'FW_GEO_ALTITUDE_diff', 'VALUE_FOB_diff', 'VALUE_FUEL_QTY_CT_diff',
       'VALUE_FUEL_QTY_FT1_diff', 'VALUE_FUEL_QTY_FT2_diff',
       'VALUE_FUEL_QTY_FT3_diff', 'VALUE_FUEL_QTY_FT4_diff',
       'VALUE_FUEL_QTY_LXT_diff', 'VALUE_FUEL_QTY_RXT_diff',
       'FLIGHT_PHASE_COUNT_diff', 'FUEL_USED_1_diff', 'TOTAL_FUEL_USED_diff',
       'VALUE_FOB_DIFF_diff', 'TOTAL_FOB_BY_QTY_diff',
       'DELTA_VFOB_VS_VFOBQTY_diff', 'ALTITUDE_DIFF_diff',
       'VALUE_FOB_MISSING_diff'],
      dtype='object')

In [78]:

pca = PCA()
pca.fit(scaled_data)

explained_var = pca.explained_variance_ratio_

cumulative_var = np.cumsum(explained_var)

explained_var, cumulative_var

(array([5.66414288e-01, 2.65355436e-01, 1.30263840e-01, 1.72156123e-02,
        5.01132360e-03, 4.23604061e-03, 3.41382827e-03, 2.54843775e-03,
        1.56717002e-03, 1.09144620e-03, 8.03988496e-04, 7.00637125e-04,
        6.27495129e-04, 4.78120266e-04, 1.69085933e-04, 9.67934108e-05,
        6.45737744e-06, 8.89314126e-31, 6.07590757e-32, 4.63270423e-32]),
 array([0.56641429, 0.83176972, 0.96203356, 0.97924918, 0.9842605 ,
        0.98849654, 0.99191037, 0.99445881, 0.99602598, 0.99711742,
        0.99792141, 0.99862205, 0.99924954, 0.99972766, 0.99989675,
        0.99999354, 1.        , 1.        , 1.        , 1.        ]))

In [79]:

n_components = len(explained_var)

variance_trace = go.Bar(
    x=list(range(1, n_components + 1)),
    y=explained_var,
    name='Explained Variance'
)

cumulative_trace = go.Scatter(
    x=list(range(1, n_components + 1)),
    y=cumulative_var,
    mode='lines',
    name='Cumulative Explained Variance'
)

layout = go.Layout(
    title='PCA - Explained and Cumulative Variance',
    xaxis=dict(title='Principal Component'),
    yaxis=dict(title='Variance Explained'),
    legend=dict(x=0.7, y=1.0),
    template='plotly_dark'
)

fig = go.Figure(data=[variance_trace, cumulative_trace], layout=layout)

fig.show()

In [80]:
pcs = pca.components_

figures = []

for i, pc in enumerate(pcs, 1):
    fig = go.Figure()
    fig.add_trace(go.Bar(x=scaled_data.columns, y=pc))
    fig.update_layout(title=f"Principal Component {i}", xaxis_title="Features", yaxis_title="PC Value", template='plotly_dark')
    figures.append(fig)
for fig in figures[0:6]:
    fig.show()

In [81]:
loadings_abs_sum = np.abs(pca.components_).sum(axis=0)

variable_index = np.argmax(loadings_abs_sum)

scaled_data.columns[variable_index]


'FLIGHT_PHASE_COUNT_diff'

In [59]:
scaled_data.columns[variable_index]

array([2.5452014 , 2.5793202 , 2.57838516, 2.68727589, 1.97367167,
       2.55988248, 2.39756441, 2.34721798, 2.41081397, 2.52825175,
       2.71452218, 2.76638288, 2.8426508 , 2.5784276 , 1.95676735,
       1.7744082 , 2.16831128, 1.81323946, 2.55952652, 2.34567897])